In [40]:
# import tabula 
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline

In [110]:
import os 
import subprocess
import re

In [244]:
base ="./Decretos Aprobados/"
i=0
prefixCols = ["evento", "fecha", "sesion", "descripcion"]
dfs = None

In [245]:
i = 0
for directorio in os.listdir(base):
    for dir2 in os.listdir(base+directorio):
        if os.path.isdir(base+directorio+"/"+dir2) == False: continue
        for filepdf in os.listdir(base+directorio+"/"+dir2):
            i+=1
            # AAA Esto es un chapuz para saltarse los primeros 1606 documentos
            #if (i<=1606): continue
            print(i, "processing file: ", base+directorio+"/"+dir2 + "/" + filepdf)
            if (filepdf.endswith("pdf") == False): continue
            #df = tabula.read_pdf(base+directorio+"/"+dir2 + "/" + filepdf, pages='all', pandas_options ={ 
            #    "names": ["Voto Emitido", "Diputado", "x1", "Partido", "x2"] })
            subpoutput = subprocess.call(["pdftotext", "-layout",  base+directorio+"/"+dir2 + "/" + filepdf , "output.txt"])
            pdftxt_f = open("./output.txt", "r")
            pdftxt = pdftxt_f.read()
            lineas = pdftxt.split("\n")
            t1 = lineas[0].split("  ")
            j = 0
            while j<100:
                if lineas[j].strip().startswith("Voto Emitido"):
                    break
                j+=1
                
            prefix = [t1[0], t1[-1], lineas[2], " ".join(lineas[3:j])]
            pdftxt_f.close()
            
            tipoVoto = None
            data = []
            for linea in lineas:
                linea = linea.strip()
                if linea.startswith("Voto Favor"):
                    tipoVoto = "Si"
                elif linea.startswith("Voto Contra"):
                    tipoVoto = "No"
                elif linea.startswith("Ausente"):
                    tipoVoto = "Aus"
                elif linea.startswith("Licencia"):
                    tipoVoto = "Lic"
                    
                if len(linea.replace(" ", ""))>12 and re.match("^\s*\d.*$", linea) is not None: 
                    row = [y for y in linea.split("  ") if y!=""]
                    row[0] = row[0][(row[0].index(" ")+1):]
                    row.append(tipoVoto)
                    data.append(row)
                else:
                    continue
                
            subprocess.call(["rm", "./output.txt"])
            
            df = pd.DataFrame(data, columns=["Apellidos", "Nombres", "Partido", "TipoVoto"])
            
            for j, col in enumerate(prefixCols):
                df[col] = prefix[j]
            df["Filename"] = base+directorio+"/"+dir2 + "/" + filepdf
            df = df.astype("category")
            if dfs is None:
                dfs = df
            else:
                dfs = pd.concat([dfs, df], 0, ignore_index=True)
                if (i%10==0):
                    print("Eporting")
                    dfs.to_csv("Votos Congreso Aprobaciones - 2012-2018.csv.gz")
            

1 processing file:  ./Decretos Aprobados/Decretos 2017/019-2017/Preámbulo.pdf
2 processing file:  ./Decretos Aprobados/Decretos 2017/019-2017/Artículo 2.pdf
3 processing file:  ./Decretos Aprobados/Decretos 2017/019-2017/Redacción Final.pdf
4 processing file:  ./Decretos Aprobados/Decretos 2017/019-2017/Tercer Debate.pdf
5 processing file:  ./Decretos Aprobados/Decretos 2017/019-2017/Artículo 1.pdf
6 processing file:  ./Decretos Aprobados/Decretos 2017/006-2017/Preámbulo.pdf
7 processing file:  ./Decretos Aprobados/Decretos 2017/006-2017/Artículo 2 y enmienda.pdf
8 processing file:  ./Decretos Aprobados/Decretos 2017/006-2017/Artículo 3.pdf
9 processing file:  ./Decretos Aprobados/Decretos 2017/006-2017/Enmienda por adición artículo 2.pdf
10 processing file:  ./Decretos Aprobados/Decretos 2017/006-2017/Redacción Final.pdf
Eporting
11 processing file:  ./Decretos Aprobados/Decretos 2017/006-2017/Artículo 1 y enmienda.pdf
12 processing file:  ./Decretos Aprobados/Decretos 2017/006-2017/A

In [246]:
dfs.to_csv("Votos Congreso Aprobaciones - 2012-2018.csv.gz")

In [264]:
dfs = dfs[dfs.TipoVoto.notna()]

In [280]:
dfs["NombreCompleto"] = (dfs.Nombres + " " + dfs.Apellidos)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [274]:
dfs["uniqueNombre"] = (dfs.Nombres + "" + dfs.Apellidos).map(lambda x: re.sub("[aeiouáéíóú\- ]", "", x.lower()) )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [272]:
pd.options.display.max_rows = 300

In [282]:
mapaNombres = dfs.groupby("uniqueNombre").NombreCompleto.first()

In [283]:
dfs["NombreCorregido"] = dfs.uniqueNombre.map(lambda x: mapaNombres[x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [299]:
dfs.to_csv("Votos Congreso Aprobaciones - 2012-2018.csv.gz")

In [300]:
conteosNombre = dfs.groupby("TipoVoto").NombreCorregido.value_counts().unstack(0) # .sort_values("Si")

In [301]:
conteosNombre["Total"] = conteosNombre.sum(1)

In [305]:
porcNombre = conteosNombre.divide(conteosNombre.Total, 0) * 100
porcNombre["TotalN"] = conteosNombre.Total

In [308]:
porcNombre.head()

TipoVoto,Aus,Lic,No,Si,Total,TotalN
NombreCorregido,,,,,,
Alfredo Augusto Rabbe Tejada,19.881557,2.284264,11.844332,65.989848,100.0,1182.0
Alicia Dolores Beltrán López,22.065379,5.126300,13.632987,59.175334,100.0,2692.0
Alsider Antonio Arias Rodríguez,10.174029,3.882195,28.781794,57.161981,100.0,747.0
Amílcar Aleksander Castillo Roca,0.338409,NaN,23.434856,76.226734,100.0,1182.0
Ana Regina Del Rosario Guzmán Sánchez,2.030457,5.245347,5.922166,86.802030,100.0,1182.0


In [311]:
conteosPartido = dfs.groupby("TipoVoto").Partido.value_counts().unstack(0) # .sort_values("Si")
conteosPartido["Total"] = conteosPartido.sum(1)
porcPx = conteosPartido.divide(conteosPartido.Total, 0) * 100
porcPx["TotalN"] = conteosPartido.Total

In [315]:
porcPx.sort_values("Aus", ascending=False).head(n=20)

TipoVoto,Aus,Lic,No,Si,Total,TotalN
Partido,,,,,,
Frente Republicano Guatemalteco,63.865546,NaN,22.689076,13.445378,100.0,238.0
Partido Republicano Institucional,61.904762,NaN,22.023810,16.071429,100.0,336.0
Partido Victoria,55.136268,1.467505,12.788260,30.607966,100.0,477.0
Bloque Legislativo Progresista -PRO-,45.590994,4.502814,20.825516,29.080675,100.0,1066.0
Independientes,34.061838,8.179227,13.047292,44.711644,100.0,13681.0
WINAQ,33.724340,19.574780,18.768328,27.932551,100.0,1364.0
Frente Republicano Guatemalteco,33.521127,NaN,14.084507,52.394366,100.0,355.0
Libertad Democrática Renovada,33.312193,4.340415,19.739971,42.607420,100.0,25228.0
Libertad Democrática Renovada,32.254812,3.375860,21.403832,42.965496,100.0,22809.0


In [319]:
dfs["uniquePx"] = dfs.Partido.map(lambda x: re.sub("[aeiouáéíóú\-\, ]", "", x.lower()) )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [320]:
mapaPx = dfs.groupby("uniquePx").Partido.first()

In [326]:
dfs.uniquePx.unique(), dfs.uniquePx.apply(lambda x: x[0:10]).unique(), dfs.uniquePx.apply(lambda x: x[0:8]).unique()

(array(['fcnncn', 'cnvrgnc', 'blqlgsltvrfrmdr', 'nddncnldl', 'blqlnzcddn',
        'ndpndnts', 'cmprmsrnvcny', 'nndlcmbncnl', 'prtddvnzdncnl',
        'nddrvlcnrncnl', 'tds', 'ncntrprgtml', 'vsncnvlrs', 'frz', 'wnq',
        'prtdnnst', 'prtdptrt', 'cmprmsrnvcnyrdn', 'nddncnldlsprnz',
        'lbrtddmcrtcrnvd', 'nddrvlcnrncnlgtmltcmz', 'blqlgsltvprgrsstpr',
        'crnnst', 'blqndpndnt', 'blqndpndntpdms', 'blqndpndnttds',
        'prtdrpblcnnsttcnl', 'grnlnzncnl', 'prtdvctr', 'frntrpblcngtmltc',
        'frntmpl', 'clcnnddncnldlsprnz–grn', 'clcnnddncnldlsprnz–grnln'],
       dtype=object),
 array(['fcnncn', 'cnvrgnc', 'blqlgsltvr', 'nddncnldl', 'blqlnzcddn',
        'ndpndnts', 'cmprmsrnvc', 'nndlcmbncn', 'prtddvnzdn', 'nddrvlcnrn',
        'tds', 'ncntrprgtm', 'vsncnvlrs', 'frz', 'wnq', 'prtdnnst',
        'prtdptrt', 'nddncnldls', 'lbrtddmcrt', 'blqlgsltvp', 'crnnst',
        'blqndpndnt', 'prtdrpblcn', 'grnlnzncnl', 'prtdvctr', 'frntrpblcn',
        'frntmpl', 'clcnnddncn'], dtype=

In [328]:
dfs["uniquePx"] = dfs.uniquePx.apply(lambda x: x[0:8])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [329]:
mapaPx = dfs.groupby("uniquePx").Partido.first()

In [330]:
dfs["PxCorregido"] = dfs.uniquePx.map(lambda x: mapaPx[x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [335]:
conteosPartido = dfs.groupby("TipoVoto").PxCorregido.value_counts().unstack(0) # .sort_values("Si")
conteosPartido["Total"] = conteosPartido.sum(1)
porcPx = conteosPartido.divide(conteosPartido.Total, 0) * 100
porcPx["TotalN"] = conteosPartido.Total

In [337]:
porcPx.sort_values("No", ascending=False)

TipoVoto,Aus,Lic,No,Si,Total,TotalN
PxCorregido,,,,,,
Coalición Unidad Nacional De La Esperanza – Gran,20.068027,NaN,36.054422,43.877551,100.0,294.0
Partido de Avanzada Nacional,7.975871,2.613941,34.199062,55.211126,100.0,5968.0
Frente Amplio,0.793651,NaN,23.015873,76.190476,100.0,126.0
CREO-UNIONISTA,7.931034,NaN,22.413793,69.655172,100.0,290.0
Libertad Democrática Renovada,32.810126,3.882424,20.530008,42.777442,100.0,48037.0
Bloque Alianza Ciudadana,16.565996,5.134228,20.419463,57.880313,100.0,17880.0
Encuentro por Guatemala,14.021949,4.085719,19.924366,61.967967,100.0,13486.0
WINAQ,22.291504,15.900234,19.173811,42.634451,100.0,2566.0
FCN-NACIÓN,20.120227,5.810276,18.163919,55.905578,100.0,47743.0


In [382]:
decretos = dfs.Filename.map(lambda x: x.split("/")[-2])

In [384]:
dfs["Decreto"] = decretos

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [369]:
leyes = dfs.descripcion.map(lambda x: ",".join(re.findall("(\d{4})", x) ))

In [374]:
leyes.unique() #.map(len).value_counts()

array(['4433', '5065', '5352', '5200', '4986', '4966', '3416', '5049',
       '5111', '5178', '5012', '5216', '5351', '5333', '5120', '5260',
       '5271', '5034,4554,5025', '4944', '5198', '', '4904,5194', '5231',
       '4432', '5080', '5056', '4791', '5023', '4204', '5189', '4428',
       '4865', '4824', '4588', '2016', '5117', '4640', '4889', '5084',
       '5020', '4777', '5151', '4334', '4972', '5102', '5131', '5021',
       '4870', '5028', '5090', '4930', '5121', '4958', '4983,4975',
       '4790', '4597', '5045', '4724', '4982', '5044', '4971', '4681',
       '4767', '5007', '4836', '4962', '4763', '4676', '4974', '4893',
       '5052', '4991', '4520', '4746', '5006', '4861', '4651', '4742',
       '4980', '4827', '4837', '4868', '4812', '4951', '4996', '4702',
       '4735', '4621', '4819', '4013', '4552', '3228', '4332', '4698',
       '4832', '4825', '4642', '4859', '4466', '4528', '4705', '4890',
       '4458', '4810', '4845', '4499', '5498', '5010', '5343', '5565',
      

In [375]:
dfs["IniciativasLeyes"] = leyes 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [378]:
dfs[dfs.IniciativasLeyes == ""].descripcion.values

array(['APROBACIÓN DE ENMIENDA POR SUSTITUCIÓN TOTAL AL ARTÍCULO 8 DEL PROYECTO DE DECRETO QUE DISPONE APROBAR LA INICIATIVA',
       'APROBACIÓN DE ENMIENDA POR SUSTITUCIÓN TOTAL AL ARTÍCULO 8 DEL PROYECTO DE DECRETO QUE DISPONE APROBAR LA INICIATIVA',
       'APROBACIÓN DE ENMIENDA POR SUSTITUCIÓN TOTAL AL ARTÍCULO 8 DEL PROYECTO DE DECRETO QUE DISPONE APROBAR LA INICIATIVA',
       ...,
       'EXTRAORDINARIO APROBACIÓN DEL ART. 1 DEL PROYECTO DE DECRETO QUE DISPONE APROBAR TRATADO DE LIBRE COMERCIO ENTRE LOS ESTADOS UNIDOS MEXICANOS Y LAS REPÚBLICAS DE COSTA RICA, EL SALVADOR, GUATEMALA, HONDURAS Y NICARAGUA',
       'EXTRAORDINARIO APROBACIÓN DEL ART. 1 DEL PROYECTO DE DECRETO QUE DISPONE APROBAR TRATADO DE LIBRE COMERCIO ENTRE LOS ESTADOS UNIDOS MEXICANOS Y LAS REPÚBLICAS DE COSTA RICA, EL SALVADOR, GUATEMALA, HONDURAS Y NICARAGUA',
       'EXTRAORDINARIO APROBACIÓN DEL ART. 1 DEL PROYECTO DE DECRETO QUE DISPONE APROBAR TRATADO DE LIBRE COMERCIO ENTRE LOS ESTADOS UNIDOS MEXICANOS

In [385]:
dfs.to_csv("Votos Congreso Aprobaciones - 2012-2018.csv.gz")